# 在数据处理流水线中使用VineyardRuntime实现高效中间数据管理

## 概述
当下的大数据/AI应用，往往需要使用端到端的流水线来实现，以下图所示的一个风控作业数据操作流为例：1首先，需要从数据库中导出订单相关数据；随后，图计算引擎会处理这些原始数据，构建 用户-商品 关系图，并通过图算法，初筛出其中隐藏的潜在作弊团伙；接下来，机器学习算法会对这些潜在团伙进行作弊归因，筛选出更准确的结果；最后这些结果会经过人工筛查，并最终做出业务处理。

![Workflow](./static/workflow.png)

在这样的场景下，我们常常会遇到如下问题：
1. 开发环境和生产环境的差异导致数据工作流的开发和调试变得复杂且低效：
数据科学家在自己的计算机上开发数据操作的操作使用 Python 代码，但是又需要在生产环境中将代码转化为他们并不熟悉的 YAML 文件从而利用 Argo、Tekton 等基于 Kubernetes 的工作流引擎，这大大降低了开发和部署效率，也带来了开发和生产环境差异性大带来的风险。
2. 需要引入新分布式存储实现中间临时数据交换，带来额外的开发、费用、运维成本：
端到端任务的子任务之间的数据交换通常依赖分布式文件系统或对象存储系统（如 HDFS、S3、OSS），这使得整个工作流需要进行大量的数据格式转换和适配工作，导致冗余的 I/O 操作，并由于中间数据的短期性，使用分布式存储系统会导致额外的成本。

3. 在大规模 Kubernetes 集群环境中的数据处理的效率问题：
在大规模的 Kubernetes 集群中，使用现有的分布式文件系统处理数据时，由于调度系统对数据的读写本地性缺乏足够的理解，并未有效地考虑到数据的位置问题，没有充分利用数据的局部性，导致在处理节点间的数据交换时，无法避免大量的数据重复拉取操作。这种操作既增加了 I/O 消耗，也降低了整体的运行效率。

![workflow with vineyard](./static/workflow_with_vineyard.png)
为了解决现有大数据/AI中的数据流操作存在的上述问题，我们结合了 Vineyard 的数据共享机制和 Fluid的数据编排能力。
1. Fluid 的 Python SDK 能够方便地对数据流进行编排，为熟悉 Python 的数据科学家提供了一种简单的方式来构建和提交以数据集操作为中心的工作流。特别地，在开发环境和云上生产环境通过一套代码进行数据流管理。
2. Vineyard 使端到端工作流中任务之间的数据共享更加高效， 通过内存映射的方式实现零拷贝数据共享，从而避免了额外的 IO 开销，这个是数据共享效率提升的关键。
3. 通过利用 Fluid 的数据亲和性调度能力，在 Pod 调度策略考虑数据写入节点的信息，从而减小数据迁移引入的网络开销，提升端到端性能。

## 代码示例

在接下来的示例中，我们将使用Fluid中的VineyardRuntime以及DataFlow功能展示如何在数据处理流水线中实现高效中间数据管理。DataFlow功能是Fluid内建提供的数据流编排能力，可将数据处理过程中的多个数据操作串联，实现简单的逻辑编排。如果希望使用更为高级的工作流编排能力，VineyardRuntime同样支持与Argo Workflow等工作流编排引擎集成使用。

### 1. 数据集准备

In [ ]:
%pip install oss2 numpy pandas

In [ ]:
import os
os.environ["OSS_ACCESS_KEY_ID"] = "<YOUR_ACCESS_KEY>"
os.environ["OSS_ACCESS_KEY_SECRET"] = "<YOUR_ACCESS_SECRET>"

In [ ]:
import numpy as np
import pandas as pd

# Fake data
num_rows = 600 * 1000
df = pd.DataFrame({
    'Id': np.random.randint(1, 100000, num_rows),
    'MSSubClass': np.random.randint(20, 201, size=num_rows),
    'LotFrontage': np.random.randint(50, 151, size=num_rows),
    'LotArea': np.random.randint(5000, 20001, size=num_rows),
    'OverallQual': np.random.randint(1, 11, size=num_rows),
    'OverallCond': np.random.randint(1, 11, size=num_rows),
    'YearBuilt': np.random.randint(1900, 2022, size=num_rows),
    'YearRemodAdd': np.random.randint(1900, 2022, size=num_rows),
    'MasVnrArea': np.random.randint(0, 1001, size=num_rows),
    'BsmtFinSF1': np.random.randint(0, 2001, size=num_rows),
    'BsmtFinSF2': np.random.randint(0, 1001, size=num_rows),
    'BsmtUnfSF': np.random.randint(0, 2001, size=num_rows),
    'TotalBsmtSF': np.random.randint(0, 3001, size=num_rows),
    '1stFlrSF': np.random.randint(500, 4001, size=num_rows),
    '2ndFlrSF': np.random.randint(0, 2001, size=num_rows),
    'LowQualFinSF': np.random.randint(0, 201, size=num_rows),
    'GrLivArea': np.random.randint(600, 5001, size=num_rows),
    'BsmtFullBath': np.random.randint(0, 4, size=num_rows),
    'BsmtHalfBath': np.random.randint(0, 3, size=num_rows),
    'FullBath': np.random.randint(0, 5, size=num_rows),
    'HalfBath': np.random.randint(0, 3, size=num_rows),
    'BedroomAbvGr': np.random.randint(0, 11, size=num_rows),
    'KitchenAbvGr': np.random.randint(0, 4, size=num_rows),
    'TotRmsAbvGrd': np.random.randint(0, 16, size=num_rows),
    'Fireplaces': np.random.randint(0, 4, size=num_rows),
    'GarageYrBlt': np.random.randint(1900, 2022, size=num_rows),
    'GarageCars': np.random.randint(0, 5, num_rows),
    'GarageArea': np.random.randint(0, 1001, num_rows),
    'WoodDeckSF': np.random.randint(0, 501, num_rows),
    'OpenPorchSF': np.random.randint(0, 301, num_rows),
    'EnclosedPorch': np.random.randint(0, 201, num_rows),
    '3SsnPorch': np.random.randint(0, 101, num_rows),
    'ScreenPorch': np.random.randint(0, 201, num_rows),
    'PoolArea': np.random.randint(0, 301, num_rows),
    'MiscVal': np.random.randint(0, 5001, num_rows),
    'TotalRooms': np.random.randint(2, 11, num_rows),
    "GarageAge": np.random.randint(1, 31, num_rows),
    "RemodAge": np.random.randint(1, 31, num_rows),
    "HouseAge": np.random.randint(1, 31, num_rows),
    "TotalBath": np.random.randint(1, 5, num_rows),
    "TotalPorchSF": np.random.randint(1, 1001, num_rows),
    "TotalSF": np.random.randint(1000, 6001, num_rows),
    "TotalArea": np.random.randint(1000, 6001, num_rows),
    'MoSold': np.random.randint(1, 13, num_rows),
    'YrSold': np.random.randint(2006, 2022, num_rows),
    'SalePrice': np.random.randint(50000, 800001, num_rows),
})

print("DataFrame initialized.")

import oss2
import io
from oss2.credentials import EnvironmentVariableCredentialsProvider
# 请将您的 OSS accessKeyID 和 accessKeySecret 分别设置成环境变量 OSS_ACCESS_KEY_ID 和 OSS_ACCESS_KEY_SECRET
auth = oss2.ProviderAuth(EnvironmentVariableCredentialsProvider())
# 请将 OSS_ENDPOINT 和 BUCKET_NAME 替换为您的 OSS Endpoint 和 Bucket
bucket = oss2.Bucket(auth, '<OSS_ENDPOINT>', '<OSS_BUCKET_NAME>')

bytes_buffer = io.BytesIO()
df.to_pickle(bytes_buffer)
bucket.put_object("df.pkl", bytes_buffer.getvalue())

### 2. 创建Fluid Dataset和VineyardRuntime

In [1]:
# Setting fluidsdk logger level to DEBUG for detailed messages
import logging
import sys
logger = logging.getLogger("fluidsdk")
stream_handler = logging.StreamHandler(sys.stdout)
stream_handler.setFormatter(logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s'))
logger.addHandler(stream_handler)
logger.setLevel(logging.DEBUG)

In [2]:
import fluid

from fluid import constants
from fluid import models

# 使用默认kubeconfig文件连接到 Fluid 控制平台，并创建 Fluid 客户端实例
client_config = fluid.ClientConfig()
fluid_client = fluid.FluidClient(client_config)

# 在default namespace下创建名为vineyard的数据集
fluid_client.create_dataset(
    dataset_name="vineyard",
)

# 获取vineyard数据集实例
dataset = fluid_client.get_dataset(dataset_name="vineyard")

# 初始化vineyard runtime的配置，并将vineyard数据集实例绑定到该runtime。
# 副本数为2，内存分别为30Gi
dataset.bind_runtime(
    runtime_type=constants.VINEYARD_RUNTIME_KIND,
    replicas=2,
    cache_capacity_GiB=30,
    cache_medium="MEM",
    wait=True
)

2024-03-08 12:08:26,145 - fluidsdk - DEBUG - Dataset "default/vineyard" created


在上述代码片段中：
- 创建 Fluid 客户端: 这段代码负责使用默认的kubeconfig文件建立与Fluid控制平台的连接，并创建一个Fluid客户端实例。
- 创建和配置 vineyard 数据集与运行时环境: 接下来，代码创建了一个名为Vineyard的数据集，然后获取该数据集实例，并初始化vineyard运行时的配置，设置副本数和内存大小，将数据集绑定到运行时环境。

### 3. 定义Fluid DataFlow

In [3]:
from kubernetes.client import models as k8s_models
# 定义任务运行模版，并挂载OSS Volume
def create_processor(process_func, packages_to_install, pip_index_url):
    extra_volumes = k8s_models.V1Volume(
                name="data",
                persistent_volume_claim=k8s_models.V1PersistentVolumeClaimVolumeSource(
                    claim_name="pvc-oss"
                )
            )
    extra_volume_mount = k8s_models.V1VolumeMount(
                name="data",
                mount_path="/data"
            )
    
    from fluid.utils import processor as processor_utils
    debug_mode = True # Setting debug_mode to True for verbose
    processor = processor_utils.make_processor_from_func(process_func, packages_to_install=packages_to_install, pip_index_url=pip_index_url, volumes=[extra_volumes], volume_mounts=[extra_volume_mount], debug_mode=debug_mode)

    return processor

在上述代码片段中：
- **创建任务模版:** 代码中封装了一个名为`create_processor`的任务模板函数，该函数接收一个Python函数对象，并自动解析Python函数对象中的代码内容，最后将代码传入作为某个容器的启动命令。`create_processor`函数还可以设置运行该函数所需的Python版本（默认为3.10版本，参考`processor_utils.make_processor_from_func`的方法签名）和PyPI依赖。容器还将在`/data`目录下挂载OSS存储数据源。

> 注意：挂载OSS存储数据源前需要在集群提前创建名为`pvc-oss`的PersistentVolumeClaim（PVC）资源，并将其绑定一个OSS类型的PersistentVolume（PV）资源上。PV资源需要指定数据准备步骤中上传的Bucket路径。

In [4]:
# 定义数据预处理脚本
def preprocess():
    from sklearn.model_selection import train_test_split

    import pandas as pd
    import vineyard
    
    df = pd.read_pickle('/data/df.pkl')
    
    # Preprocess Data
    df = df.drop(df[(df['GrLivArea']>4800)].index)
    X = df.drop('SalePrice', axis=1)  # Features
    y = df['SalePrice']  # Target variable
    
    del df
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    
    del X, y
    
    vineyard.put(X_train, name="x_train", persist=True)
    vineyard.put(X_test, name="x_test", persist=True)
    vineyard.put(y_train, name="y_train", persist=True)
    vineyard.put(y_test, name="y_test", persist=True)


def train():
    from sklearn.linear_model import LinearRegression

    import joblib
    import pandas as pd
    import vineyard

    x_train_data = vineyard.get(name="x_train", fetch=True)
    y_train_data = vineyard.get(name="y_train", fetch=True)

    model = LinearRegression()
    model.fit(x_train_data, y_train_data)

    joblib.dump(model, '/data/model.pkl')


def test():
    from sklearn.linear_model import LinearRegression
    from sklearn.metrics import mean_squared_error

    import vineyard
    import joblib
    import pandas as pd

    x_test_data = vineyard.get(name="x_test", fetch=True)
    y_test_data = vineyard.get(name="y_test", fetch=True)

    model = joblib.load("/data/model.pkl")
    y_pred = model.predict(x_test_data)

    err = mean_squared_error(y_test_data, y_pred)

    with open('/data/output.txt', 'a') as f:
        f.write(str(err))


packages_to_install = ["numpy", "pandas", "pyarrow", "requests", "vineyard", "scikit-learn==1.4.0", "joblib==1.3.2"]
pip_index_url = "https://pypi.tuna.tsinghua.edu.cn/simple"

preprocess_processor = create_processor(preprocess, packages_to_install, pip_index_url)
train_processor = create_processor(train, packages_to_install, pip_index_url)
test_processor = create_processor(test, packages_to_install, pip_index_url)

上述代码片段分别定义了数据处理流水线中的三个步骤：数据预处理、模型训练和模型测试。这三个步骤对应的Python函数传入`create_processor`函数以被封装为三个processor。

In [5]:
# 创建线性回归模型的任务工作流：数据预处理 -> 模型训练 -> 模型测试
# 下列的挂载路径"/var/run"是vineyard配置文件的默认路径
flow = dataset.process(processor=preprocess_processor, dataset_mountpath="/var/run") \
              .process(processor=train_processor, dataset_mountpath="/var/run") \
              .process(processor=test_processor, dataset_mountpath="/var/run")

In [6]:
# 将线性回归模型的数据处理任务工作流提交，并等待其运行完成
run = flow.run(run_id="linear-regression-with-vineyard")
run.wait()

2024-03-08 12:13:09,983 - fluidsdk - INFO - DataProcess linear-regression-with-vineyard-step1 completed
2024-03-08 12:15:26,417 - fluidsdk - INFO - DataProcess linear-regression-with-vineyard-step2 completed
2024-03-08 12:17:39,682 - fluidsdk - INFO - DataProcess linear-regression-with-vineyard-step3 completed


### 4. 资源清理

In [7]:
# 清理所有资源
dataset.clean_up(wait=True)